### Recommendations with MovieTweetings: Most Popular Recommendation

Now that you have created the necessary columns we will be using throughout the rest of the lesson on creating recommendations, let's get started with the first of our recommendations.

To get started, read in the libraries and the two datasets you will be using throughout the lesson using the code below.


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tests as t

%matplotlib inline

# Read in the datasets
movies = pd.read_csv('movies_clean.csv')
reviews = pd.read_csv('reviews_clean.csv')
del movies['Unnamed: 0']
del reviews['Unnamed: 0']

#### 1. How To Find The Most Popular Movies

For this notebook, we have a single task.  The task is that no matter the user, we need to provide a list of the recommendations based on simply the most popular items.

For this task, we will consider what is "most popular" based on the following criteria:

* A movie with the highest average rating is considered best
* With ties, movies that have more ratings are better
* A movie must have a minimum of 5 ratings to be considered among the best movies
* If movies are tied in their average rating and number of ratings, the ranking is determined by the movie that is the most recent rating

With these criteria, the goal for this notebook is to take a **user_id** and provide back the **n_top** recommendations.  Use the function below as the scaffolding that will be used for all the future recommendations as well.

In [2]:
reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 863866 entries, 0 to 863865
Data columns (total 5 columns):
user_id      863866 non-null int64
movie_id     863866 non-null int64
rating       863866 non-null int64
timestamp    863866 non-null int64
date         863866 non-null object
dtypes: int64(4), object(1)
memory usage: 33.0+ MB


In [5]:
def ranked_movies_df(movies,reviews):
    movie_ratings = reviews.groupby('movie_id')['rating']
    avg_ratings = movie_ratings.mean()
    num_ratings = movie_ratings.count()
    last_ratings = pd.DataFrame(reviews.groupby('movie_id').max()['date'])
    last_ratings.columns = ['lasting_ratings']
    
    ratings_count_df = pd.DataFrame({"avg_ratings":avg_ratings,"num_ratings":num_ratings})
    ratings_count_df = ratings_count_df.join(last_ratings)
    
    movies_recs = movies.set_index("movie_id").join(ratings_count_df)
    ranked_movies = movies_recs.sort_values(['avg_ratings',"num_ratings","lasting_ratings"],ascending = False)
    
    ranked_movies = ranked_movies[ranked_movies['num_ratings'] > 4]
    
    return ranked_movies

In [6]:
ranked_movies = ranked_movies_df(movies,reviews)

In [7]:
def popular_recommendations(user_id, n_top,ranked_movies):
    '''
    INPUT:
    user_id - the user_id of the individual you are making recommendations for
    n_top - an integer of the number recommendations you want back
    OUTPUT:
    top_movies - a list of the n_top recommended movies by movie title in order best to worst
    '''
    # Do stuff
    top_movies = list(ranked_movies["movie"][:n_top])
    
    return top_movies # a list of the n_top movies as recommended

Usint the three criteria above, you should be able to put together the above function.  If you feel confident in your solution, check the results of your function against our solution. On the next page, you can see a walkthrough and you can of course get the solution by looking at the solution notebook available in this workspace.  

In [8]:
# Put your solutions for each of the cases here

# Top 20 movies recommended for id 1

recs_20_for_1 = popular_recommendations(user_id= "1" ,n_top=20,ranked_movies=ranked_movies)# Your solution list here

# Top 5 movies recommended for id 53968
recs_5_for_53968 = popular_recommendations(user_id= "53968" ,n_top=5,ranked_movies=ranked_movies) # Your solution list here

# Top 100 movies recommended for id 70000
recs_100_for_70000 = popular_recommendations(user_id= "70000" ,n_top=100,ranked_movies=ranked_movies) # Your solution list here

# Top 35 movies recommended for id 43
recs_35_for_43 = popular_recommendations(user_id= "43" ,n_top=35,ranked_movies=ranked_movies) # Your solution list here



In [9]:
### You Should Not Need To Modify Anything In This Cell
ranked_movies = t.create_ranked_df(movies, reviews) # only run this once - it is not fast

# check 1 
assert t.popular_recommendations('1', 20, ranked_movies) == recs_20_for_1,  "The first check failed..."
# check 2
assert t.popular_recommendations('53968', 5, ranked_movies) == recs_5_for_53968,  "The second check failed..."
# check 3
assert t.popular_recommendations('70000', 100, ranked_movies) == recs_100_for_70000,  "The third check failed..."
# check 4
assert t.popular_recommendations('43', 35, ranked_movies) == recs_35_for_43,  "The fourth check failed..."

print("If you got here, looks like you are good to go!  Nice job!")

If you got here, looks like you are good to go!  Nice job!


**Notice:** This wasn't the only way we could have determined the "top rated" movies.  You can imagine that in keeping track of trending news or trending social events, you would likely want to create a time window from the current time, and then pull the articles in the most recent time frame.  There are always going to be some subjective decisions to be made.  

If you find that no one is paying any attention to your most popular recommendations, then it might be time to find a new way to recommend, which is what the next parts of the lesson should prepare us to do!


### Part II: Adding Filters

Now that you have created a function to give back the **n_top** movies, let's make it a bit more robust.  Add arguments that will act as filters for the movie **year** and **genre**.  

Use the cells below to adjust your existing function to allow for **year** and **genre** arguments as **lists** of **strings**.  Then your ending results are filtered to only movies within the lists of provided years and genres (as `or` conditions).  If no list is provided, there should be no filter applied.

You can adjust other necessary inputs as necessary to retrieve the final results you are looking for!

Try writing a few tests against the test function in our test function.  Below returns the top 20 movies for user 1 based on the specified year and genre filters.  Does yours return the same? 

```
t.popular_recs_filtered('1', 20, ranked_movies, years=['2015', '2016', '2017', '2018'], genres=['History'])
```

In [10]:
ranked_movies.head()

,movie,genre,date,1800's,1900's,2000's,Thriller,Music,Musical,Adult,...,Horror,Action,Drama,News,Mystery,Family,Adventure,avg_rating,num_ratings,last_rating
movie_id,,,,,,,,,,,,,,,,,,,,,
4921860,MSG 2 the Messenger (2015),Comedy|Drama|Fantasy|Horror,2015,0,0,1,1,1,1,1,...,1,1,1,1,1,1,1,10.0,48,2016-08-14-17-16-50
5262972,Avengers: Age of Ultron Parody (2015),Short|Comedy,2015,0,0,1,1,1,1,1,...,1,1,1,1,1,1,1,10.0,28,2016-01-08-00-44-43
6662050,Five Minutes (2017),Short|Comedy,2017,0,0,1,1,1,1,1,...,1,1,1,1,1,1,1,10.0,22,2019-04-20-22-29-19
2737018,Selam (2013),Drama|Romance,2013,0,0,1,1,1,1,1,...,1,1,1,1,1,1,1,10.0,10,2015-05-10-22-56-01
5804314,Let There Be Light (2017),Drama,2017,0,0,1,1,1,1,1,...,1,1,1,1,1,1,1,10.0,7,2019-12-25-16-27-47


In [45]:
def filter1(df,years):
    #years = list(years)
    df1 = df[df['date'].isin(years)]
    return df1

In [47]:
filter1(ranked_movies,[2010,2011,2015,2017])

,movie,genre,date,1800's,1900's,2000's,Thriller,Music,Musical,Adult,...,Horror,Action,Drama,News,Mystery,Family,Adventure,avg_rating,num_ratings,last_rating
movie_id,,,,,,,,,,,,,,,,,,,,,
4921860,MSG 2 the Messenger (2015),Comedy|Drama|Fantasy|Horror,2015,0,0,1,1,1,1,1,...,1,1,1,1,1,1,1,10.000000,48,2016-08-14-17-16-50
5262972,Avengers: Age of Ultron Parody (2015),Short|Comedy,2015,0,0,1,1,1,1,1,...,1,1,1,1,1,1,1,10.000000,28,2016-01-08-00-44-43
6662050,Five Minutes (2017),Short|Comedy,2017,0,0,1,1,1,1,1,...,1,1,1,1,1,1,1,10.000000,22,2019-04-20-22-29-19
5804314,Let There Be Light (2017),Drama,2017,0,0,1,1,1,1,1,...,1,1,1,1,1,1,1,10.000000,7,2019-12-25-16-27-47
4448444,Make Like a Dog (2015),Short|Comedy|Drama,2015,0,0,1,1,1,1,1,...,1,1,1,1,1,1,1,10.000000,5,2017-09-09-13-51-48
2059318,Third Contact (2011),Drama|Mystery|Sci-Fi|Thriller,2011,0,0,1,1,1,1,1,...,1,1,1,1,1,1,1,10.000000,5,2014-02-13-15-48-31
6798422,Agnelli (2017),Documentary,2017,0,0,1,1,1,1,1,...,1,1,1,1,1,1,1,9.800000,5,2018-05-15-22-39-34
1629443,Foster (2011),Comedy|Drama|Family,2011,0,0,1,1,1,1,1,...,1,1,1,1,1,1,1,9.800000,5,2014-03-15-23-37-57
5732482,Hans Zimmer Live on Tour (2017),Documentary|Music,2017,0,0,1,1,1,1,1,...,1,1,1,1,1,1,1,9.600000,5,2019-09-01-19-58-06


In [51]:
ranked_movies[['Sci-Fi','History']].sum(axis=1)

movie_id
4921860     2
5262972     2
6662050     2
2737018     2
5804314     2
2560840     2
2219210     2
9882084     2
4448444     2
5131914     2
2059318     2
1431149     2
5512872     2
4148400     2
9536832     2
6798422     2
111341      2
1629443     2
2592910     2
2396421     2
5732482     2
6054758     2
5134588     2
363473      2
2265179     2
5323386     2
8902990     2
2357788     2
1826550     2
73440       2
           ..
2006801     2
3283792     2
5988370     2
5669936     2
2325518     2
1540767     2
3551400     2
2357489     2
185183      2
829176      2
3104304     2
2622826     2
3561236     2
3036740     2
7086706     2
1098327     2
2147365     2
93300       2
105585      2
2573750     2
4743562     2
1213644     2
60666       2
3588588     2
4475992     2
10367276    2
3108604     2
3187378     2
4458206     2
7923374     2
Length: 11538, dtype: int64

In [41]:
ranked_movies[ranked_movies['date'].isin(["2010","2011","2012"])]

,movie,genre,date,1800's,1900's,2000's,Thriller,Music,Musical,Adult,...,Horror,Action,Drama,News,Mystery,Family,Adventure,avg_rating,num_ratings,last_rating
movie_id,,,,,,,,,,,,,,,,,,,,,
2219210,Crawl Bitch Crawl (2012),Horror|Sci-Fi|Thriller,2012,0,0,1,1,1,1,1,...,1,1,1,1,1,1,1,10.000000,6,2013-07-22-23-30-52
2059318,Third Contact (2011),Drama|Mystery|Sci-Fi|Thriller,2011,0,0,1,1,1,1,1,...,1,1,1,1,1,1,1,10.000000,5,2014-02-13-15-48-31
1629443,Foster (2011),Comedy|Drama|Family,2011,0,0,1,1,1,1,1,...,1,1,1,1,1,1,1,9.800000,5,2014-03-15-23-37-57
2366308,Everything or Nothing: The Untold Story of 007...,Documentary,2012,0,0,1,1,1,1,1,...,1,1,1,1,1,1,1,9.400000,5,2015-08-16-06-47-36
2077886,The Phantom of the Opera at the Royal Albert H...,Drama|Music|Musical|Romance|Thriller,2011,0,0,1,1,1,1,1,...,1,1,1,1,1,1,1,9.333333,9,2018-05-13-23-18-29
1857724,Hidden Colors (2011),Documentary,2011,0,0,1,1,1,1,1,...,1,1,1,1,1,1,1,9.333333,6,2016-12-28-17-17-53
2373324,Men at Lunch (2012),Documentary|Mystery,2012,0,0,1,1,1,1,1,...,1,1,1,1,1,1,1,9.285714,7,2014-06-18-14-15-25
1198101,Kites (2010),Action|Drama|Romance|Thriller,2010,0,0,1,1,1,1,1,...,1,1,1,1,1,1,1,9.200000,5,2017-09-16-17-30-31
1934205,Ce que le jour doit à la nuit (2012),Drama|Romance,2012,0,0,1,1,1,1,1,...,1,1,1,1,1,1,1,9.107143,28,2019-08-06-05-35-51


In [ ]:
## t.popular_recs_filtered('1', 20, ranked_movies, years=['2015', '2016', '2017', '2018'], genres=['History'])

In [56]:
def popular_recs_filtered(user_id, n_top,ranked_movies,years,genres):
    '''
    INPUT:
    user_id - the user_id of the individual you are making recommendations for
    n_top - an integer of the number recommendations you want back
    OUTPUT:
    top_movies - a list of the n_top recommended movies by movie title in order best to worst
    '''
    # Do stuff
    ranked_movies_years = ranked_movies[ranked_movies['date'].isin(years)]
    ranked_movies_genres = ranked_movies_years[genres].sum(axis=1)
    ranked_movies = ranked_movies_years.loc[ranked_movies_genres > 0,:]
    top_movies = list(ranked_movies["movie"][:n_top])
    
    return top_movies # a list of the n_top movies as recommended

In [60]:
top_20 = popular_recs_filtered('1', 20, ranked_movies, years=['2015', '2016', '2017', '2018'], genres=['History'])

In [61]:
top_20

['MSG 2 the Messenger (2015)',
 'Avengers: Age of Ultron Parody (2015)',
 'Five Minutes (2017)',
 'Let There Be Light (2017)',
 'Make Like a Dog (2015)',
 'Pandorica (2016)',
 'Be Somebody (2016)',
 'Agnelli (2017)',
 'Hans Zimmer Live on Tour (2017)',
 'Kirik Party (2016)',
 'Chasing Asylum (2016)',
 'Poshter Girl (2016)',
 'Dag II (2016)',
 'Take the Ball, Pass the Ball (2018)',
 'Chasing Coral (2017)',
 'Kuroshitsuji: Book of the Atlantic (2017)',
 'Cyborgs: Heroes Never Die (2017)',
 'Hannah Gadsby: Nanette (2018)',
 "Hillary's America: The Secret History of the Democratic Party (2016)",
 'Coldplay: A Head Full of Dreams (2018)']